import pandas as pd
import math
## starts with a spatial join, pick an attribute as the waterway/passage line label (currently "JOIN_FID")

In [2]:
all_pass = pd.read_csv("01FEB_WAMS/2021_SF_WWY.csv")

In [ ]:
## temp hold for arcpro layer select
# import pandas as pd
# from arcgis.features import GeoAccessor, GeoSeriesAccessor

# SEDF = pd.DataFrame.spatial.from_featureclass("T2021_Track_WGS84_RAW")

# rename_dict = {x:x.split(".")[1] for x in SEDF if x != "SHAPE"}

# SEDF.rename(columns=rename_dict, inplace=True)
# SEDF = SEDF[["MMSI", "VesselType", "Length", "Join_Count", "NAME"]]

# SEDF.to_csv("C:/Users/NABuch/Desktop/Makah/01FEB_WAMS/2021_SF_WWY.csv")

In [3]:
all_pass = all_pass[all_pass["Join_Count"] > 0]

In [4]:
transit_df = all_pass.groupby(by=['NAME', 'VesselType']).count().reset_index()
unique_df = all_pass.groupby(by=['NAME', 'VesselType', 'MMSI']).count().reset_index()
transit_df.head()

,NAME,VesselType,Unnamed: 0,MMSI,Length,Join_Count
0,BODEGA BAY,30.0,114,114,112,114
1,BODEGA BAY,36.0,26,26,26,26
2,BODEGA BAY,37.0,70,70,58,70
3,BODEGA BAY,90.0,18,18,6,18
4,CARQUINEZ STRAIT,0.0,33,33,10,33


In [5]:
new_df = pd.DataFrame(columns=["Waterway", "Vessel_Type", "Total_Transits", "Unique_Vessels"])
temp_type = pd.unique(all_pass["VesselType"])
temp_label = pd.unique(all_pass["NAME"])


In [6]:

for x in range(len(temp_label)):
    for i in range(len(temp_type)):
        new_df = new_df.append({"Waterway": temp_label[x], "Vessel_Type": temp_type[i]}, ignore_index=True)

new_df.head()


,Waterway,Vessel_Type,Total_Transits,Unique_Vessels
0,SACRAMENTO DEEP WATER SHIP CHANNEL (DWSC),0.0,NaN,NaN
1,SACRAMENTO DEEP WATER SHIP CHANNEL (DWSC),37.0,NaN,NaN
2,SACRAMENTO DEEP WATER SHIP CHANNEL (DWSC),74.0,NaN,NaN
3,SACRAMENTO DEEP WATER SHIP CHANNEL (DWSC),70.0,NaN,NaN
4,SACRAMENTO DEEP WATER SHIP CHANNEL (DWSC),36.0,NaN,NaN


In [7]:
for x in new_df.index:
    temp_transit = transit_df[(transit_df["VesselType"] == new_df["Vessel_Type"][x]) & (transit_df["NAME"] == new_df["Waterway"][x])].reset_index(drop=True)
    temp_transit.head()
    if len(temp_transit) > 0:
        new_df.iloc[x, 2] = temp_transit["MMSI"][0]
    else: new_df.iloc[x, 2] = 0
    temp_unique = unique_df[(unique_df["VesselType"] == new_df["Vessel_Type"][x]) & (unique_df["NAME"] == new_df["Waterway"][x])].reset_index(drop=True)
    if len(temp_unique) > 0:
        new_df.iloc[x, 3] = len(temp_unique["MMSI"])
    else: new_df.iloc[x, 3] = 0
    

In [8]:
Aggregate_temp = new_df.groupby(by="Waterway").sum().reset_index()
Aggregate_temp = Aggregate_temp["Waterway"]
Aggregate_temp.head()

0                BODEGA BAY
1          CARQUINEZ STRAIT
2      CORTE MADERA CHANNEL
3    EAST SAN FRANCISCO BAY
4         GULF OF FARALLONS
Name: Waterway, dtype: object

In [9]:
Passage_line_df = pd.DataFrame(columns=["Waterway", "Max_Len", "Unique_Fishing", "Transits_Fishing", "Unique_Tug_Tow", "Transits_Tug_Tow", "Unique_Recreational", "Transits_Recreational", "Unique_Passenger", "Transits_Passenger", "Unique_Cargo", "Transits_Cargo", "Unique_Tanker", "Transits_Tanker", "Unique_Other", "Transits_Other", "Unique_Total", "Transits_Total"])
Passage_line_df["Waterway"] = Aggregate_temp
Passage_line_df["Max_Len"] = 0
Passage_line_df["Unique_Fishing"] = 0
Passage_line_df["Unique_Tug_Tow"] = 0
Passage_line_df["Unique_Recreational"] = 0
Passage_line_df["Unique_Passenger"] = 0
Passage_line_df["Unique_Cargo"] = 0
Passage_line_df["Unique_Tanker"] = 0
Passage_line_df["Unique_Other"] = 0
Passage_line_df["Unique_Total"] = 0
Passage_line_df["Transits_Fishing"] = 0
Passage_line_df["Transits_Tug_Tow"] = 0
Passage_line_df["Transits_Recreational"] = 0
Passage_line_df["Transits_Passenger"] = 0
Passage_line_df["Transits_Cargo"] = 0
Passage_line_df["Transits_Tanker"] = 0
Passage_line_df["Transits_Other"] = 0
Passage_line_df["Transits_Total"] = 0


In [11]:
for i in Passage_line_df.index:
    maxim = [x for x in all_pass[all_pass["NAME"] == Passage_line_df["Waterway"][i]]["Length"] if math.isnan(x) == False]
    if len(maxim) > 0:
        Passage_line_df.loc[i, "Max_Len"] = max(maxim)
    else:
        Passage_line_df.loc[i, "Max_Len"] = ""
    for x in new_df.index:
        if new_df["Waterway"][x] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Total"] += new_df["Unique_Vessels"][x]
            Passage_line_df.loc[i, "Transits_Total"] += new_df["Total_Transits"][x]
        if new_df["Vessel_Type"][x] == 30 and new_df["Waterway"][x] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Fishing"] += new_df["Unique_Vessels"][x]
            Passage_line_df.loc[i, "Transits_Fishing"] += new_df["Total_Transits"][x]
        elif (new_df["Vessel_Type"][x] == 31 or new_df["Vessel_Type"][x] == 32 or new_df["Vessel_Type"][x] == 52) and new_df["Waterway"][x] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Tug_Tow"] += new_df["Unique_Vessels"][x]
            Passage_line_df.loc[i, "Transits_Tug_Tow"] += new_df["Total_Transits"][x]
        elif (new_df["Vessel_Type"][x] == 36 or new_df["Vessel_Type"][x] == 37) and new_df["Waterway"][x] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Recreational"] += new_df["Unique_Vessels"][x]
            Passage_line_df.loc[i, "Transits_Recreational"] += new_df["Total_Transits"][x]
        elif (new_df["Vessel_Type"][x] >=60 and new_df["Vessel_Type"][x] <= 69) and new_df["Waterway"][x] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Passenger"] += new_df["Unique_Vessels"][x]
            Passage_line_df.loc[i, "Transits_Passenger"] += new_df["Total_Transits"][x]
        elif (new_df["Vessel_Type"][x] >=70 and new_df["Vessel_Type"][x] <= 79) and new_df["Waterway"][x] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Cargo"] += new_df["Unique_Vessels"][x]
            Passage_line_df.loc[i, "Transits_Cargo"] += new_df["Total_Transits"][x]
        elif (new_df["Vessel_Type"][x] >=80 and new_df["Vessel_Type"][x] <= 89) and new_df["Waterway"][x] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Tanker"] += new_df["Unique_Vessels"][x]
            Passage_line_df.loc[i, "Transits_Tanker"] += new_df["Total_Transits"][x]
            
    Passage_line_df.loc[i, "Unique_Other"] = Passage_line_df["Unique_Total"][i]
    Passage_line_df.loc[i, "Transits_Other"] = Passage_line_df["Transits_Total"][i]
    
    for n in new_df.index:

        if new_df["Vessel_Type"][n] == 30 and new_df["Waterway"][n] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Other"] -= new_df["Unique_Vessels"][n]
            Passage_line_df.loc[i, "Transits_Other"] -= new_df["Total_Transits"][n]
        elif (new_df["Vessel_Type"][n] == 31 or new_df["Vessel_Type"][n] == 32 or new_df["Vessel_Type"][n] == 52) and new_df["Waterway"][n] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Other"] -= new_df["Unique_Vessels"][n]
            Passage_line_df.loc[i, "Transits_Other"] -= new_df["Total_Transits"][n]
        elif (new_df["Vessel_Type"][n] == 36 or new_df["Vessel_Type"][n] == 37) and new_df["Waterway"][n] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Other"] -= new_df["Unique_Vessels"][n]
            Passage_line_df.loc[i, "Transits_Other"] -= new_df["Total_Transits"][n]
        elif (new_df["Vessel_Type"][n] >=60 and new_df["Vessel_Type"][n] <= 69) and new_df["Waterway"][n] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Other"] -= new_df["Unique_Vessels"][n]
            Passage_line_df.loc[i, "Transits_Other"] -= new_df["Total_Transits"][n]
        elif (new_df["Vessel_Type"][n] >=70 and new_df["Vessel_Type"][n] <= 79) and new_df["Waterway"][n] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Other"] -= new_df["Unique_Vessels"][n]
            Passage_line_df.loc[i, "Transits_Other"] -= new_df["Total_Transits"][n]
        elif (new_df["Vessel_Type"][n] >=80 and new_df["Vessel_Type"][n] <= 89) and new_df["Waterway"][n] == Passage_line_df["Waterway"][i]:
            Passage_line_df.loc[i, "Unique_Other"] -= new_df["Unique_Vessels"][n]
            Passage_line_df.loc[i, "Transits_Other"] -= new_df["Total_Transits"][n]

In [12]:
Passage_line_df.to_csv("2021_track_SF.csv")